<a href="https://colab.research.google.com/github/jephianlin/NLP-demos/blob/main/NSYSU/W01-use-pretrained-models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is adapted by [Haowen Jiang](https://howard-haowen.rohan.tw/) from [this one](https://github.com/nlptown/nlp-notebooks/blob/master/NLP%20with%20pretrained%20models%20-%20spaCy%20and%20StanfordNLP.ipynb) included in the [nlptown
/nlp-notebooks](https://github.com/nlptown/nlp-notebooks) repo. It is meant for the 2022 [NLP Workshop at NSYSU](https://howard-haowen.rohan.tw/NLP-demos/nsysu_workshop).

In [ ]:
from datetime import date

today = date.today()
print("Last updated:", today)

# NLP with pretrained models - spaCy and StanfordNLP

In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_md
!python -m spacy download zh_core_web_md

![](https://spacy.io/pipeline-fde48da9b43661abcdf62ab70a546d71.svg)

In [ ]:
import spacy
from IPython.display import HTML, display
import tabulate

In [ ]:
spacy.info()

- To get you started, play with [this Web App](https://share.streamlit.io/howard-haowen/spacy-streamlit/app.py) that I created, which is powered by spaCy.

## English NLP

In [ ]:
en = spacy.load("en_core_web_md")

In [ ]:
text = ("Donald John Trump (born June 14, 1946) is the 45th and current president of "
        "the United States.  Before entering politics, he was a businessman and television personality.")
print(text)

In [ ]:
doc_en = en(text)

First spaCy splits your document into sentences, and the sentences in tokens.

- doc.sents = generator
- type(doc.sents)


In [ ]:
list(doc_en.sents)

In [ ]:
tokens = [token.text for token in doc_en]
print(tokens)

In addition, spaCy also identifies a number of linguistic features for every token. The most basic of these are the lemma, and two types of parts-of-speech tags: the `pos_` attribute contains the [Universal POS tags](https://universaldependencies.org/u/pos/) from the [Universal Dependencies](https://universaldependencies.org/), while the `tag_` attribute contains more fine-grained, language-specific part-of-speech tags.

In [ ]:
features = [[t.orth_, t.lemma_, t.pos_, t.tag_] for t in doc_en]
display(HTML(tabulate.tabulate(features, tablefmt='html')))

Next, spaCy also offers pre-trained models for named entity recognition. Their results can be found on the `ent_iob_` and `ent_type` attributes. The `ent_type` attribute tells us what type of entity the token refers to. In the English models, these entity types follow the [OntoNotes standard](https://catalog.ldc.upenn.edu/docs/LDC2013T19/OntoNotes-Release-5.0.pdf). In our example, we see that `Donald John Trump` refers to a person, `June 14, 1946` to a date, `45th` to an ordinal number, and `the United States` to a geo-political entity (GPE).

The letters on the `ent_iob_` attribute give the position of the token in the entity. `O` means the token is outside of an entity, `B` means the token is at the beginning of an entity, and `I` means it is inside an entity (at any position except for the beginning). In this way, we can tell apart several entities of the same type that immediately follow each other. Together these letters form the so-called `BIO` tagging scheme. There are other tagging schemes, such as `BILUO`, which also has letters for the last position and single (unique) tokens in an entity, but the BIO scheme gives you all the information you need.  

In [ ]:
entities = [(t.orth_, t.ent_iob_, t.ent_type_) for t in doc_en]
display(HTML(tabulate.tabulate(entities, tablefmt='html')))

You can also access the entities directly on the `ents` attribute of the document:

In [ ]:
print([(ent.text, ent.label_) for ent in doc_en.ents])

spaCy also contains a dependency parser, which analyzes the grammatical relations between the tokens.

In [ ]:
syntax = [[token.text, token.dep_, token.head.text ] for token in doc_en]
display(HTML(tabulate.tabulate(syntax, tablefmt='html')))

Finally, the English spaCy model contains a morphological parser.

In [ ]:
features = [[t.text, str(t.morph)] for t in doc_en]
display(HTML(tabulate.tabulate(features, tablefmt='html')))

## Multilingual NLP

SpaCy doesn't only have models for English, but also for many other languages.

In [ ]:
zh = spacy.load("zh_core_web_md")

In [ ]:
text_zh = "中山大學人文暨科技跨領域學士學位學程助理教授宋世祥表示，2021年聖誕節假期期間，師生舉辦「街頭玩童～鹽埕兒童街區遊戲日」成果展。活動中可看見學生運用贊助單位瑞儀教育基金會致贈的廢棄木棧板，製作了6具兒童創意遊具，一方面展示學習成果，也希望藉此呼籲高雄民眾重視兒童的遊戲權。"
doc_zh = zh(text_zh)

The tokens in the Chinese document have the same attributes as those in the English one. Take care, however, because the functionality of the models can differ across languages. Here are three main differences between the English and the Chinese model:

- The Chinese model does not offer lemmatization: the `text` attribute is identical to the `orth_` attribute.

In [ ]:
list(doc_zh.sents)

In [ ]:
tok_text = [tok.text for tok in doc_zh]
tok_orth = [tok.orth_ for tok in doc_zh]
print(tok_text)
print(tok_orth)

- The Chinese model has a very different fine-grained part-of-speech tags on the `tag_` attribute.

In [ ]:
for tok in list(doc_zh.sents)[1]:
    print(f"{tok.text} >>> {tok.pos_}")

In [ ]:
for tok in list(doc_zh.sents)[1]:
    print(f"{tok.text} >>> {tok.tag_} | {spacy.explain(tok.tag_)}")

- The Chinese model has different entity types (PER, LOC and ORG) than the English one.

This is a result of the training corpora that were used to build the models, whose annotation guidelines may be very different.

In [ ]:
info = [(t.text, t.pos_, t.tag_, t.ent_iob_, t.ent_type_) for t in doc_zh]
display(HTML(tabulate.tabulate(info, tablefmt='html')))

## Visualization

In [ ]:
from spacy import displacy

In [ ]:
displacy.render(doc_zh, style='ent',jupyter=True, options={'distance':130})

In [ ]:
text = "我想要三份2號餐"
doc = zh(text)
displacy.render(doc, style='dep',jupyter=True, options={'distance':130})

## DataFrame + spaCy = dframcy

In [ ]:
!pip install dframcy

In [ ]:
from dframcy import DframCy

In [ ]:
nlp = spacy.load('zh_core_web_md')
dframcy = DframCy(nlp)
doc = dframcy.nlp(text_zh)
annotation_dataframe = dframcy.to_dataframe(doc)
annotation_dataframe

Once annotations are stored as a DataFrame object, filtering can be easily done by leveraging the power of `pandas` syntax.

In [ ]:
nn_filt = annotation_dataframe['token_tag_'] == 'NN'
dobj_filt = annotation_dataframe['token_dep_'] == 'dobj'
annotation_dataframe[(nn_filt) & dobj_filt]

## Vectors

In [ ]:
doc = zh("教授")
tok = doc[0]
tok.vector

In [ ]:
tok.vector.shape

In [ ]:
word_1 = nlp.vocab["高興"]
word_2 = nlp.vocab["高雄"]
word_3 = nlp.vocab["開心"]
word_1_word_2 = word_1.similarity(word_2)
word_1_word_3 = word_1.similarity(word_3)
print(f"Distance btn Word 1 and 2: {word_1_word_2}")
print(f"Distance btn Word 1 and 3: {word_1_word_3}")

- Cosine similarity

![](https://datascience-enthusiast.com/figures/cosine_sim.png)

- Formula for calculating cosine similarity between two vectors

![](https://miro.medium.com/max/1400/1*LfW66-WsYkFqWc4XYJbEJg.png)

## StanfordNLP

Another library whose functionality overlaps with that of spaCy is StanfordNLP. [StanfordNLP](https://stanfordnlp.github.io/stanfordnlp/), not to be confused with Stanford's Java [CoreNLP](https://stanfordnlp.github.io/CoreNLP/) library, is a [Python library](https://github.com/stanfordnlp/stanfordnlp) built on top of PyTorch that offers a fully neural pipeline with tokenization (including multi-word units), lemmatization, part-of-speech tagging (including morphological features) and dependency parsing. These components were built and trained for the [CoNLL-2018 shared task](https://nlp.stanford.edu/pubs/qi2018universal.pdf). There are no named entities, but the quality of the dependency parsing is state of the art. On top of that, it also offers a Python interface to CoreNLP.

**`stanfordnlp` has been renamed to `stanza`.**

In [32]:
!pip install stanza

  Created wheel for emoji: filename=emoji-2.7.0-py2.py3-none-any.whl size=356563 sha256=d09aaa8666d376e9e35b949302cff36ee8b5830d5171ba869dd0ef3993ecda53
  Stored in directory: /root/.cache/pip/wheels/41/11/48/5df0b9727d5669c9174a141134f10304d1d78a3b89a4676f3d
Successfully built emoji


In [33]:
import stanza

In [34]:
stanza.download("zh-hant")

INFO:stanza:Downloading default packages for language: zh-hant (Traditional_Chinese) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.


In [35]:
stf_nlp = stanza.Pipeline('zh-hant')

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: zh-hant (Traditional_Chinese):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| pos       | gsd     |
| lemma     | gsd     |
| depparse  | gsd     |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Done loading processors!


In [36]:
text_zh = "中山大學人文暨科技跨領域學士學位學程助理教授宋世祥表示，2021年聖誕節假期期間，師生舉辦「街頭玩童～鹽埕兒童街區遊戲日」成果展。活動中可看見學生運用贊助單位瑞儀教育基金會致贈的廢棄木棧板，製作了6具兒童創意遊具，一方面展示學習成果，也希望藉此呼籲高雄民眾重視兒童的遊戲權。"
doc = stf_nlp(text_zh)
type(doc)

stanza.models.common.doc.Document

Different models often produce different tokenization results, which in turn would have impact on POS and DEP tagging.

- Here're the results based on StandfordNLP.

In [37]:
for i, sent in enumerate(doc.sentences):
    print("[Sentence {}]".format(i+1))
    for word in sent.words:
        print("{:12s}\t{:12s}\t{:6s}\t{:d}\t{:12s}".format(\
              word.text, word.lemma, word.pos, word.head, word.deprel))
    print("")

[Sentence 1]
中山          	中山          	PROPN 	3	nmod        
大學          	大學          	NOUN  	3	nmod        
人文          	人文          	NOUN  	12	nmod        
暨           	暨           	CCONJ 	6	cc          
科技          	科技          	NOUN  	6	compound    
跨           	跨           	PART  	3	conj        
領域          	領域          	NOUN  	12	nmod        
學士          	學士          	NOUN  	12	nmod        
學位          	學位          	NOUN  	12	nmod        
學程          	學程          	NOUN  	12	nmod        
助理          	助理          	NOUN  	12	nmod        
教授          	教授          	NOUN  	15	nsubj       
宋           	宋           	PROPN 	12	appos       
世祥          	世祥          	PROPN 	13	flat:name   
表示          	表示          	VERB  	0	root        
，           	，           	PUNCT 	15	punct       
2021        	2021        	NUM   	18	nummod      
年           	年           	NOUN  	22	clf         
聖誕          	聖誕          	PROPN 	20	compound    
節           	節           	PART  	22	nmod        
假期          	

# Assignment


## Analyze Enlgish

- Input: any English news article of your choice
- Ouput:
    - A list of unique lemmas of all verbs in lower case
    - A list of unique tuples of (NER text, NER label)




In [38]:
# Change this to any other article of your choice.

en_input = """
Taipei, April 7 (CNA) Health and Welfare Minister Chen Shih-chung (陳時中) said Thursday that COVID-19 contact tracing has been partially suspended in Taiwan and a new disease control model is being put in place, amid a rise in domestic cases.

The immediate suspension of contract tracing applies only to travelers who test positive for COVID-19 in Taiwan, either on arrival at the airport or during mandatory quarantine, Chen said.

That decision was made in a bid to free up resources to monitor the growing number of domestic COVID-19 cases, he said at a press briefing, after he reported 531 new cases -- 382 domestically transmitted and 149 imported.

Chen said contact tracing on new imported cases will only be done if any of them are believed to be linked to COVID-19 clusters at quarantine hotels or quarantine centers in Taiwan.

Prior to Thursday, Taiwan had been reporting its contact tracing information on imported COVID-19 cases via the World Health Organization's International Health Regulations (IHR) mechanism, he said.

Regarding the recent daily rise in domestic infections, Chen said the current goal is to bring the situation under control, even though it is impossible to achieve zero new domestic cases at this time.

Despite the recent spike, the daily number of domestic COVID-19 cases in Taiwan is still low compared to many other countries, he said, citing as an example the 534 new cases per 100,000 population reported in South Korea on Tuesday.

Once people in Taiwan stick together and do their part to prevent the spread of the virus, the situation will be manageable, Chen said.

Based on the trajectory of COVID-19 Omicron outbreaks observed in many other countries around the world, he said, it is likely that the infections in Taiwan will peak in a month or two.

"We do not expect the outbreak to stop growing now, but we hope it will rise slowly, so that Taiwan's medical capacity will not be overloaded," Chen said.

Meanwhile, earlier in the day, the Cabinet announced that Taiwan was adopting a new model for the control of COVID-19 infections.

Under the "new Taiwan model," the country has let go of its goal to achieve zero COVID-19 cases, but this does not mean allowing the pandemic go unmanaged, Cabinet spokesman Lo Ping-cheng (羅秉成) said, citing Premier Su Tseng-chang (蘇貞昌).

In a meeting earlier with Ministry of Health and Welfare (MOHW) officials, Premier Su said that as Taiwan moves towards a new stage of epidemic prevention, he hopes that the central and local governments will work together to gradually open up the country, in the interests of its people and economy, according to Lo.

In a report presented to the Cabinet on Thursday, the MOHW said Taiwan will continue to actively manage the COVID-19 situation, while steadily opening up its borders, in consideration of national economic factors and the people's livelihood.
"""

In [39]:
# Start by turning a text into a spaCy Doc object
# en_doc = en(en_input)

In [40]:
#===Write your code below and save the output as `verbs`.===#


# verbs =

In [41]:
#===Write your code below and save the output as `en_ents`.===#


# en_ents =

## Analyze Chinese

- Input 1: any Chinese news article from Taiwan media of your choice
- Ouput 1:
    - A list of unique tokens except for punctuations
    - A list of unique tuples of (NER text, NER label)

In [42]:
# Change this to any other article of your choice.

zh_input = """
本土個案昨增三八二例再創新高，確診案例遍及十九縣市，境外增一四九例，單日破五百案例。

中央疫情指揮中心指揮官陳時中表示，個案數還會往上升，預估一至兩個月內達最高峰，疫情將持續到六月底且還不會到尾聲；他也首度鬆口「清零不可能」，未來將走向與病毒共存，下周擬試辦「輕症在家」隔離，同時也將調整停課標準。

擬以居家快篩取代停課
國內確診個案上升，全國累積十四縣市共一三九所校園停課，各縣市對畢業旅行、戶外教學是否取消標準不一，教師團體認為政府應該明確表態，否則會造成校園恐慌或影響學生受教權。陳時中表示，未來必然走向與病毒共存，下周將與教育部檢討停課標準，縮小匡列範圍，並在合理範圍以居家快篩來取代停課。

疫情一至兩個月達高峰
指揮中心比照韓國、紐西蘭及香港疫情發展，推估未來一至兩個月確診案例將飆至最高峰，陳時中表示，目前Omicron確診數仍算低，但規模難以預估，未來單日恐超過一千五百例，屆時致死率、個案數飆高或疫情高峰下不來，社會將承擔不起，因此仍須積極因應，朝「緩坡上升」方向努力。

輕症在家指引至今沒譜
總統蔡英文日前宣布防疫以「減災」為目標避免醫療量能超載，「輕症在家」隔離為其中配套，單日確診數若達一千五百人將啟動。指揮中心下周將擬定「輕症在家照護指引」，如設醫療遠距平台、送藥、戶政及警政系統聯繫、關懷中心運作等，若未遵守隔離規定將有罰則，並因應地方疫情升溫和醫療量能吃緊，將從新北試辦。

立委賴惠員、蔣萬安昨於衛環委員會質詢時，質疑各地方早已喊話希望指揮中心相關指引快出來，但至今連社區溝通、徵求試辦的地方政府意願統統沒有譜。

台北市長柯文哲昨晚也在臉書表示「北市防疫旅館量能告急」，因每天確診人數不斷攀升，近期有近萬名移工入境，幾乎把北市的防疫旅館量能占滿。北市已發出徵召令，徵用加強版防疫專責旅館，讓輕症、低危險確診者入住。

總統府、監察院傳確診者
本土疫情多點爆發，公務機關包括總統府、監察院、台北市議會都傳出有確診者，北中南八大行業確診人數也驟增，未來場所是否新增禁令，指揮中心將再討論。

境外移入昨增一四九例個案，有七十八例為航班落地採檢陽性，越南有十七例居冠。指揮中心表示，目前落地採檢陽性率約落在四％到五％間，即日起越南航空、越捷航空及越竹航空等三航空公司班機，增加「搭機前六小時內抗原快篩報告」才可入境。
"""

In [43]:
# Start by turning a text into a spaCy Doc object
# zh_doc = zh(zh_input)

In [44]:
#===Write your code below and save the output as `zh_toks`.===#

# zh_toks =

In [45]:
#===Write your code below and save the output as `zh_ents`.===#


# zh_ents =

- Input 2: Simplified version of Input 1 (Use `opencc` to do the conversion.)
- Ouput 2:
    - A list of unique tokens except for punctuations
    - A list of unique tuples of (NER text, NER label)

In [46]:
!pip install opencc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.3/778.3 kB 9.5 MB/s eta 0:00:00


In [47]:
import opencc

In [48]:
converter = opencc.OpenCC('t2s.json')
sim_zh_input = converter.convert(zh_input)
sim_zh_input

'\n本土个案昨增三八二例再创新高，确诊案例遍及十九县市，境外增一四九例，单日破五百案例。\n\n中央疫情指挥中心指挥官陈时中表示，个案数还会往上升，预估一至两个月内达最高峰，疫情将持续到六月底且还不会到尾声；他也首度松口「清零不可能」，未来将走向与病毒共存，下周拟试办「轻症在家」隔离，同时也将调整停课标准。\n\n拟以居家快筛取代停课\n国内确诊个案上升，全国累积十四县市共一三九所校园停课，各县市对毕业旅行、户外教学是否取消标准不一，教师团体认为政府应该明确表态，否则会造成校园恐慌或影响学生受教权。陈时中表示，未来必然走向与病毒共存，下周将与教育部检讨停课标准，缩小匡列范围，并在合理范围以居家快筛来取代停课。\n\n疫情一至两个月达高峰\n指挥中心比照韩国、纽西兰及香港疫情发展，推估未来一至两个月确诊案例将飙至最高峰，陈时中表示，目前Omicron确诊数仍算低，但规模难以预估，未来单日恐超过一千五百例，届时致死率、个案数飙高或疫情高峰下不来，社会将承担不起，因此仍须积极因应，朝「缓坡上升」方向努力。\n\n轻症在家指引至今没谱\n总统蔡英文日前宣布防疫以「减灾」为目标避免医疗量能超载，「轻症在家」隔离为其中配套，单日确诊数若达一千五百人将启动。指挥中心下周将拟定「轻症在家照护指引」，如设医疗远距平台、送药、户政及警政系统联系、关怀中心运作等，若未遵守隔离规定将有罚则，并因应地方疫情升温和医疗量能吃紧，将从新北试办。\n\n立委赖惠员、蒋万安昨于卫环委员会质询时，质疑各地方早已喊话希望指挥中心相关指引快出来，但至今连社区沟通、征求试办的地方政府意愿统统没有谱。\n\n台北市长柯文哲昨晚也在脸书表示「北市防疫旅馆量能告急」，因每天确诊人数不断攀升，近期有近万名移工入境，几乎把北市的防疫旅馆量能占满。北市已发出征召令，征用加强版防疫专责旅馆，让轻症、低危险确诊者入住。\n\n总统府、监察院传确诊者\n本土疫情多点爆发，公务机关包括总统府、监察院、台北市议会都传出有确诊者，北中南八大行业确诊人数也骤增，未来场所是否新增禁令，指挥中心将再讨论。\n\n境外移入昨增一四九例个案，有七十八例为航班落地采检阳性，越南有十七例居冠。指挥中心表示，目前落地采检阳性率约落在四％到五％间，即日起越南航空、越捷航空及越竹航空等三航空公司班机，增加「搭机前六小时内抗原快筛报告」才可入境。\n

In [49]:
# Start by turning a text into a spaCy Doc object
# sim_zh_doc = zh(sim_zh_input)

In [50]:
#===Write your code below and save the output as `sim_zh_toks`.===#


# sim_zh_toks =

Evaluate whether `zh_toks` is equal to `sim_zh_toks`.

In [51]:
# zh_toks == sim_zh_toks

In [52]:
#===Write your code below and save the output as `sim_zh_ents`.===#


# sim_zh_ents =

Evaluate whether `zh_ents` is equal to `sim_zh_ents`.

In [53]:
# zh_ents == sim_zh_ents